In [ ]:
# !pip install ydata_profiling

In [ ]:
# pip install --upgrade visions

In [ ]:
import pandas as pd
import numpy as np
import optuna

import sklearn as sk
import lightgbm

import optiver2023


# import pandas_profiling as pp
# from ydata_profiling import ProfileReport, compare

In [ ]:
train_data = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')

In [ ]:
# df_report = ProfileReport(train_data, title = 'ydata_report')
# df_report

In [ ]:
train_data.imbalance_size.isna().sum()

In [ ]:
def inspect_columns(df):
    result = pd.DataFrame({
        'unique': df.nunique() == len(df),
        'cardinality': df.nunique(),
        'with_null': df.isna().any(),
        'null_pct': round((df.isnull().sum()/ len(df))*100, 2),
        'null_num': df.isnull().sum(),
        '1st_now': df.iloc[0],
        'random_row':df.iloc[np.random.randint(low = 0, high = len(df))],
        'last_row':df.iloc[-1],
        'dtype': df.dtypes
        
    })
    return result

In [ ]:
train_data
#5291000-5237980 = 53020 missing

In [ ]:
inspect_columns(train_data)

In [ ]:
(train_data['target'] == 0).sum()

In [ ]:
#imbalance size 最小值与最大值差距较大、0值较多、缺失值220
# reference_price 缺失值220
# matched_size 缺失值220
# far_price/near_price 300seconds以后
# ask_price/bid_price 缺失值220
# target 缺失值88 0值3336
# wap缺失值220

In [ ]:
train_data[train_data['imbalance_size'].isna()][['stock_id', 'date_id']].drop_duplicates().values

In [ ]:
#缺失的220即为上述四只股票各存在一天数据的缺失

In [ ]:
#stock[131,101,158,19]

In [ ]:
all_stock_id = np.sort(train_data["stock_id"].unique())
all_date_id = np.sort(train_data["date_id"].unique())

In [ ]:
multi_index = pd.MultiIndex.from_product(
    iterables = [all_date_id, all_stock_id],
    names = ["date_id", "stock_id"]
)

missing_data = (train_data
                .loc[:, ["date_id", "stock_id"]]
                .assign(in_train = True)
                .set_index(["date_id", "stock_id"])
                .merge(multi_index.to_frame(), how="right", left_index = True, right_index = True)
                .query("in_train.isna()")
                .set_index("stock_id", drop = True)
                ["date_id"]
               )
missing_data

In [ ]:
missing_data.index.unique()

In [ ]:
missing_data.unique()

In [ ]:
missing_data.groupby("stock_id").count().sort_values()

In [ ]:
X = train_data.query("target.notna()").drop(['row_id', 'time_id'], axis = 1)
to_remove = [
    [131, 35],
    [101, 328],
    [158, 388],
    [19, 438]
]

to_remove_df = pd.DataFrame(to_remove, columns=['stock_id', 'date_id'])

X = X.merge(to_remove_df, on=['stock_id', 'date_id'], how='left', indicator=True)
X = X[X['_merge'] == 'left_only'].drop(columns=['_merge'])

y = X.pop("target")
X.head(5)

In [ ]:
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def objective_lgb(trial, X, Y):
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mae_scores = []

    param = {
        'objective': 'regression_l1', 
        'metrics': 'mae',  
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.03),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'num_leaves': trial.suggest_int('num_leaves', 10, 150),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 5),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 5),
        'subsample_for_bin': 200000,
        'min_split_gain': 0.0,
        'importance_type': 'split', 
        'random_state': 42, 
        'device': 'gpu',  
        'force_row_wise': True,  
        'verbose': -1 
    }

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

        model = lgb.LGBMRegressor(**param)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mae_scores.append(mae)

    return sum(mae_scores) / len(mae_scores)

In [ ]:
def optimize_lgbm(X, y, n_trials=100):
    study = optuna.create_study(direction='minimize')  
    study.optimize(lambda trial: objective_lgb(trial, X, y), n_trials=n_trials)
    
    print("最佳参数：", study.best_params)
    print("最低 MAE：", study.best_value)
    
    return study

In [ ]:
study = optimize_lgbm(X, y, n_trials=100)

best_params = study.best_params

In [ ]:
# best_params = {
#                  'objective': 'regression_l1', 
#                 'metrics': 'mae',  
#                 'boosting_type': 'gbdt',
#                 'learning_rate': 0.029980482801499182, 
#                'max_depth': 11, 
#                'num_leaves': 141, 
#                'min_child_samples': 42, 
#                'subsample': 0.9416185637958379, 
#                'colsample_bytree': 0.9657715386759944, 
#                'n_estimators': 967, 
#                'lambda_l1': 4.95430532734202, 
#                'lambda_l2': 4.212756955941557,
#                'importance_type': 'split', 
#                 'random_state': 42, 
#                 'device': 'gpu',  
#                 'force_row_wise': True,  
#                 'verbose': -1 
# }

In [ ]:
Training = True
if Training:
    model = lgb.LGBMRegressor(**best_params)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model.fit(X_train, y_train)


    y_pred = model.predict(X_test)
    final_mae = mean_absolute_error(y_test, y_pred)
    print(f"最终模型的 MAE: {final_mae}")
    print(f"最佳参数:{best_params}")

In [ ]:
# env = optiver2023.make_env()
# iter_test = env.iter_test()

# SEED = 42

# counter = 0
# for (test, revealed_targets, sample_prediction) in iter_test:
#     if counter == 0:
#         print(test.head(3))
#         print(revealed_targets.head(3))
#         print(sample_prediction.head(3))
#     test = test[X_train.columns]
#     sample_prediction['target'] = model.predict(test)
#     env.predict(sample_prediction)
#     counter += 1